In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
sb.set()
import scipy.optimize as optimization
import datetime as dt
from datetime import timedelta
from matplotlib import rcParams
rcParams['figure.figsize'] = 10,6
import math
import scipy.stats
import matplotlib.pyplot as plt
import time
from ipywidgets import Checkbox
import pandas_datareader as pdr
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# apend path to definiotion
import sys
sys.path.append('../../ModernPortfolio_project/DEFINITIONS')

# Import all necessery function
import StockSelection as stock
import DataPreparation as prep
import MarkowitzModel as m_model
import Rebalancing as reb
import Summary as summ

### 1. Select stock you are intrested in.

We can distinguish general markets as:
- equities
- commodities
- precious metals
- cryptocurrencies
- bonds

If you want to built long-term portfolio which is resistant to prosperity cycles you should consider selecting at least one stock for each market types. Thanks that you will elimitane systematic risk and let your welth grow independently. 

My modern portfolio will not only help you diversify your portefl but also find the best proportion of your assets to achieve the highest possible rate of return with the lowest possible investment risk.  


If you do not know the stocks by tickers check short description [here](http://localhost:8888/notebooks/ModernPortfolio_project/TryModernPortfolioYourself/1.CheckFoundAveliableToAnalysis.ipynb)

In [ ]:
# Create checkbox listo to choese
SPX_= Checkbox(description = 'CSPX.L')
BTC_ = Checkbox(description = 'BTC-USD')
IEF_ = Checkbox(description = 'IEF')
FILL_ = Checkbox(description = 'FILL')
PALL_ = Checkbox(description = 'PALL')
FXI_ = Checkbox(description = 'FXI')
GMF_ = Checkbox(description = 'GMF')
EXS1_ = Checkbox(description = 'EXS1.DE')
VGK_ = Checkbox(description = 'VGK')
DBC_ = Checkbox(description = 'DBC')
BNO_ = Checkbox(description = 'BNO')
PICK_ = Checkbox(description = 'PICK')
SGOL_ = Checkbox(description = 'SGOL')
PPLT_ = Checkbox(description = 'PPLT')
SIVR_ = Checkbox(description = 'SIVR')
BNO_ = Checkbox(description = 'BNO')
period = Checkbox(description = 'Over choesen period')
margin = Checkbox(description = 'Over choesen margin')
quaterly = Checkbox(description = 'quaterly')
yearly = Checkbox(description = 'yearly')
full_DR = Checkbox(description = '365')
busieness_DR = Checkbox(description = '252')
HighestSR = Checkbox(description = 'Highest Sharp Ratio')
MyWallet = Checkbox(description = 'My Choice')

In [ ]:
tick = display('Equity', SPX_, FXI_, GMF_, EXS1_, VGK_, \
        'Comodity', DBC_, FILL_, BNO_, PICK_, \
        'Precious metals', SGOL_, PPLT_, SIVR_, PALL_,\
       'Crypto', BTC_, \
       'Bonds', IEF_)

In [ ]:
tickers = stock.choese_stock(SPX_, FXI_, GMF_, EXS1_, VGK_,DBC_, FILL_, BNO_, PICK_,SGOL_, PPLT_, SIVR_, PALL_,BTC_,IEF_)

### 1.1 Add more tickers by hand

In [ ]:
# First we must check if ticker exist or if has not been choesen already
new_tickers = [''] # FORMAT eg. ticker = ['AMZN', 'TSLA', 'GM']

In [ ]:
tickers = stock.add_stock_by_hand(tickers, new_tickers)

In [ ]:
tickers

### 2. Select period of analysis

Remember that last aveliable end date is day before today

In [ ]:
pd.to_datetime(dt.datetime.today())

In [ ]:
start = '' # FORMAT: 'yyyy-mm-dd'
end = ''   # FORMAT: 'yyyy-mm-dd'

### 3. Check if given period is aveliable

Some assets may be listed later than the period you have selected. At this stage we want to check if the date you have selected is available. If NOT - the algorithm will tell you which market it is and when the earliest data is available.

In [ ]:
new_start = stock.find_first_date_from_choesen_stosks(start, end, tickers)

### 4. Check day of the week

Usually stocks are listed in weekdays, so we nedd to be sure that date that been choesen is not listed at weekend. Algo will find the best fitted day for You. If start date is wrong it will move forward to find aveliable date. If end date is wrong it will take previous date aveliable

Start date

In [ ]:
start = stock.check_start_date(new_start)
start

End date

In [ ]:
end = stock.check_end_date(end)
end

### 5. Prepare data

As known traditional markets are listed only in weekdays, but in my modern portfolio model you can choose cryptocarecy stocks also, where days off does not exist. In order for next steps of analysis, all stocks must be in the same date range. I decided not to cut cryptocurrency persion, but instead I added the closing prices to traditional markets for weekend periods (Sat, Sun). The values for weekend is last listed value at week day (friday). In the future I will add funtionality which let's you make that decision.

##### 5.1 Chceck quote type

We must check quote type of all assets, becouse listing frequency could be different for some of them

In [ ]:
freq, qType = prep.check_quote_type(tickers, start, end)

In [ ]:
tickers, tic_type = prep.sort_tickers(qType, tickers)
tic_type

In [ ]:
tickers

##### 5.2 Select frequency if needed

If you selected one or more asstes that are cryptocurrencies you must make decision now. As known traditional markets like Equity, comodity are listed in 252 days per year, becouse markets are open only at busienss day. Unfortunetly, cryptocurencies are listed 24/7 which means that they are listed every single day (356 days) per year. You can choose to work on 365 days or 252 days frequency and algo will prepare data for you.

**365:** If you select full date range per year, algo will add missing dates to traditional markets. It will just repeat Fridays listing vales for Saturdays and Sundays.

**252:** If you select business day range, algo will cut weekends listing for any cryptocuriency you have in your wallet

**Note:** If you do not see checkbox that let you choose frequency, it means that you do not have any crypto in your wallet

In [ ]:
full_DR, busieness_DR = prep.select_freq(full_DR, busieness_DR, freq)

In [ ]:
choesen_freq = prep.choese_freq(full_DR, busieness_DR, freq)

##### 5.3 Create prepared dataframe

Base on your choice, algo will handle missing values and prepare your data for future analysis

In [ ]:
df, n = prep.createDF_with_choesen_ferq(choesen_freq, tickers, start, end, prep.createPrepDataFrame, prep.PasteMissingRows, prep.create_data_without_weekends, prep.cut_weekends)

### 6. Markowitz Portfolio symulation

Here we simulate thousands of portfolio with different weights of stocks. For eg. You could invest in chosen stocks with all equal weights, so if you selected 'BTC', 'SPX500', 'FILL and 'ETL' your equal weights will be [0.25, 0.25, 0.25, 0.25] multiplied by cash you are going to invest. Markowitz portfolio symulate portfolios with different wages for egx. [1,0,0,0], wchih mean that you invest all money in BTC then [0.9, 0.1,0,0] meaning of 90% for BTC ant 10 % for SPX etc. 


The process will be repeated x times. X mean iterations, where the minimum should be 20 000, but it depend on period and quantity of stock you selected before. The simulation might take a while, but the tips are: 
- if it took less than 10 second you should increase iterations,
- if 30 secnond pass and you still have not result, just decrease iterations.

**Choese iterations:**

In [ ]:
iterations = 30000 # You can change it

In [ ]:
results, returns, vol, weights = m_model.symulatePortfoliosWeights(df, iterations, n, m_model.calculate_N, m_model.calculate_log_rets, m_model.gen_weights, m_model.calculate_returns, m_model.calculate_volatility)

### See results of your simulation on the plot

The chart describe expected return and volatility for given weights of assets. As you can see the higher return is, the higher volatility you can expect. The main thing is to keep good balance between this two values, trying to take maximum profit 

In [ ]:
m_model.createResultsPlot(results)

### 7. Sharp Ratio

We simuluate thousends of portfolios, but **which one is the best?** To anserw that question you should be awere of **Sharp Ratio**, which is calculated by dividind expected annual return by expected annual volotality for each portfolio. The one with the highest Sharp Ratio has the best proportion of risk/reward indicator. Let's check your Sharp Ratio then!

In [ ]:
maxSR, expected_vol, expected_ret, new_weights = m_model.findHighestSharpRatio(results);

In [ ]:
df.columns

##### See Sharp Ratio on plot

In [ ]:
m_model.createResultsPlot2(results, expected_vol, expected_ret)

### 8. Find the best portfolio by risk appetite

All investors has slightly differen risk appetite. The more you want to earn, the more risk u must take. Some investor could take higher risk in order to get higher profit/return. 

**Expected Volatility** describe in average how your investment will change annualy. It's also called standard deviation. 

**Example:** Let's say that your anual log return is 10% and in same way the volatility is 15%. It means that before you gain 10% profit, your portfolio will go up and down in average of 15%. So it could go up 20%, then downs 10%, then up and so on untill the investment end.

**Note:** You need to be awere that here we work on logaritmic return, but usually more important is simple returns. Becouse of that you must take into account the fact that volotality calculated by log returns might be lower than simple one. If you do not know what to choese, just write whatever you think and come back again after you get to the end of analysis. I am sure that it will be more clear to You then.

In [ ]:
# Example of how to wtire value
# 10% - 0.1
# 35% - 0.35
max_vol_yearly = 0.00 # 0.00

In [ ]:
my_rets, my_vol, sharpRatio, bestWeights = m_model.findBestWeightsByMaxVol(results, max_vol_yearly)

In [ ]:
m_model.createResultsPlot3(results, expected_vol, expected_ret, my_vol, my_rets)

In [ ]:
summary = m_model.create_summary(maxSR, sharpRatio, expected_vol, my_vol, expected_ret, my_rets, new_weights, bestWeights)
summary

**Select portfolio You want to analyze**

In [ ]:
display(HighestSR, MyWallet)

In [ ]:
tickers = m_model.choese_wallet(HighestSR, MyWallet)

###### Weights of your portfolio

In [ ]:
weights = m_model.create_weightsDataFrame(summary, tickers, df)
weights

### 9. Historical performence of your portfolio


Write your investment amount and check how your portfolio would perform over period you choese previosly.

In [ ]:
investmentAmt = # Choose how much u want to invest (int)

In [ ]:
cum_returns, buyAmt = reb.prepare_data(investmentAmt, df, weights)

In [ ]:
reb.comparePortfolios_Plot(investmentAmt, cum_returns, max_y = 1.2, freq=20);

### 10. Proportion of your portfolio over time

As you know some of stocks might grow faster than another. That is why the weights you started with may change. Let's check weights of assets in portfolio over analysis period!

In [ ]:
proportions = reb.calculate_proportions(cum_returns, df, weights)

In [ ]:
reb.plot_proportions(proportions)

**Note:** If y-axis on the chart is too thick, increase freq paramiter, which as a default is 5

If chart above does not presents horizontal straight lines, that means that your wallets lost weights established at the beginning

### 11. Rebalanceing

Markowitz model calculated annual returns and volotality base on daily log returns. Obviously it does not take into account the cummulated returns, which is good, becouse it helps maintain objectivity. So, if we want to create portfolio base on markowitz model we must keep portfolio weights at a fixed level at all times.

Unfortunetly, there is no point to rebalance our portfolio whenever the weights sligtly change, mainly becouse of transaction cost and taxes. So we presumed that rebalancing should not be done more then one time in Quater (3 months) and not less than once a Year.

**Select how would you like to rebalance your portfolio**

1. Period: It just check the proportion of your portfolio by choesen period for eg. every quater or every year passed
2. Margin: You can select margin for proportions of your wallet. For eg. if you celect margin of 20% (0.2), it will rebalance your portfolio if any of your asset's proportion exceed the margin (grow more than 20% of its assemed proportion or drop lower then 20% of assumed proportion)

In [ ]:
display(period, margin)

In [ ]:
choesen_method = reb.choese_rebalancing_method(period, margin)

**Select next paramiters**
1. Period: You can rebalance your portfolio every quater or year
2. Margin: You can set margin whatever you like. Remember it must be positive number. Choose something beetwen 0.2 and 0.5, which means 20% and 50%

In [ ]:
selection = reb.display_next_options(choesen_method, quaterly, yearly)

**In this step algo will create rebalanced portfolio by choesen methon and paramiters**

In [ ]:
new_proportions, portfolio_value = reb.run_selected_rebalancing_method(choesen_method, weights, buyAmt, cum_returns, proportions, n, quaterly, yearly, df, reb.rebalancing_model_over_period, reb.rebalancing_model_over_margin, selection, reb.calc_portfolio_value_by_date, reb.calc_portfolio_value_by_row, reb.calculate_new_proportions, reb.repair_rebalancing_dates, reb.choese_rebalancing_frquency, reb.create_dates_for_rebalancing)

**See results on the plot** and comapare them to previous plot, where you did not make any rebalancing

In [ ]:
reb.plot_proportions(new_proportions)

If the graph above does not remaind you Heart Rate Variability then you should consider increasing the frequency of rebalancing or just go to the next step and check the summary

In [ ]:
freq = 5

In [ ]:
reb.compare_balance_portfolio_vs_previous(investmentAmt, portfolio_value, cum_returns, freq=freq) 

**Note:** If y-axis on the chart is too thick, increase freq paramiter, which as a default is 5

In [ ]:
all_portfolios = summ.create_table_with_all_portfolios(cum_returns, portfolio_value)

### 12. Summary

Below you can see table with summary of 3 portfolios:
- equal weights, where all assets haave the same value in the begining
- your portfolio, where all assets have previously selected weights in the begining
- your balanced portfolio, where all assets have fixed proportions in each rebalancing period 

**Max dropdowns** show how much each portfolio drop for given period

In [ ]:
summ.dropdown_summary(all_portfolios, n)

**Returns** is an average/annual profit investor make by each portfolio

In [ ]:
summ.returns_summary(all_portfolios, investmentAmt, n)

**Note_1:** If Max dropdowns for your portfolio are too hight you can:

- select more stocks to your portfolio in setp nr. 1
- decrease max volotality in step nr. 8
- change the method of rebalancing
- change rebalancing frequency for 'quaterly' if you haven't done already in step nr. 11
- decrease margin for second rebalancing method